# HF Transformers 核心模块学习：Pipelines

**Pipelines**（管道）是使用模型进行推理的一种简单易上手的方式。

这些管道是抽象了 Transformers 库中大部分复杂代码的对象，提供了一个专门用于多种任务的简单API，包括**命名实体识别、掩码语言建模、情感分析、特征提取和问答**等。

**通俗解释**  
Pipelines 就像是一条简单的“生产线”，把模型加载、数据预处理、推理以及结果处理的复杂工作都封装起来。这样，无论是做情感分析、问答、命名实体识别还是图像分类，都只需要调用一行代码，省去了繁琐的配置和调试步骤。

**常用任务表格**

| Modality                    | Task                              | Description                                                         | Pipeline API                                         |
| --------------------------- | --------------------------------- | ------------------------------------------------------------------- | ---------------------------------------------------- |
| Audio                       | Audio classification              | 为音频文件分配一个标签                                              | `pipeline(task="audio-classification")`              |
|                             | Automatic speech recognition      | 将音频文件中的语音转换为文本                                          | `pipeline(task="automatic-speech-recognition")`      |
| Computer vision             | Image classification              | 为图像分配一个标签                                                  | `pipeline(task="image-classification")`              |
|                             | Object detection                  | 预测图像中目标对象的边界框和类别                                      | `pipeline(task="object-detection")`                  |
|                             | Image segmentation                | 为图像中每个像素分配标签（支持语义、全景和实例分割）                   | `pipeline(task="image-segmentation")`                |
|                             | Zero-shot image classification    | 对未见过的类别进行图像分类（最新常用功能）                             | `pipeline(task="zero-shot-image-classification")`    |
| Natural language processing | Text classification (sentiment)   | 为给定文本序列分配一个标签（如情感分析）                              | `pipeline(task="sentiment-analysis")`                |
|                             | Token classification (NER)        | 为文本中每个 token 分配标签（如人、组织、地点等命名实体识别）            | `pipeline(task="ner")`                               |
|                             | Question answering                | 根据上下文和问题，在文本中提取答案                                    | `pipeline(task="question-answering")`                |
|                             | Summarization                     | 为文本或文档生成摘要                                                 | `pipeline(task="summarization")`                     |
|                             | Translation                       | 将文本从一种语言翻译为另一种语言                                      | `pipeline(task="translation")`                       |
|                             | Text generation                   | 根据给定提示生成自然语言文本（最新常用功能）                           | `pipeline(task="text-generation")`                   |
|                             | Fill-mask                         | 根据上下文预测被掩盖的词语（最新常用功能）                             | `pipeline(task="fill-mask")`                         |
|                             | Zero-shot classification          | 对文本进行未见过类别的分类（最新常用功能）                             | `pipeline(task="zero-shot-classification")`          |
| Multimodal                | Document question answering       | 根据给定文档和问题回答文档相关的问题                                  | `pipeline(task="document-question-answering")`       |
|                           | Visual Question Answering         | 给定图像和问题，回答有关图像的问题                                   | `pipeline(task="vqa")`                               |
|                           | Image captioning                  | 为图像生成描述性文字（最新常用功能）                                  | `pipeline(task="image-to-text")`                     |

- 在模型列表中，页面左侧可以看到所有支持的模型和任务：https://huggingface.co/models
- 选中任意一个模型，如 https://huggingface.co/deepseek-ai/DeepSeek-R1 ，有对应的Pipeline任务，通常在第一个。
- Pipelines 已支持的完整任务列表：https://huggingface.co/docs/transformers/task_summary


## Pipeline API

**Pipeline API** 是对所有其他可用管道的包装。它可以像任何其他管道一样实例化，它将模型加载、数据预处理、推理和结果处理等复杂步骤全部封装起来，从而使 AI 推理的调用变得像调用普通函数一样直观，既保留了专业性又让使用变得简单易懂。

![](docs/images/pipeline_func.png)

---

### 为什么使用 Pipeline API？

- **简单易用**：只需几行代码，就能完成从输入到输出的整个流程。
- **降低学习成本**：你不需要深入理解模型内部的工作机制，就能实现文本分类、问答、图像识别等任务。
- **封装复杂性**：内部已经帮你处理好了模型加载、数据转换和结果解释等步骤。

---

### 使用 Pipeline API 实现 Text Classification 任务

**Text classification**(文本分类)与任何模态中的分类任务一样，文本分类将一个文本序列（可以是句子级别、段落或者整篇文章）标记为预定义的类别集合之一。文本分类有许多实际应用，其中包括：

- **情感分析**：判断文本是表达积极情绪还是消极情绪，帮助在政治、金融、市场等领域做出决策。
- **内容分类**：根据主题对文本进行分类，如过滤新闻、社交媒体内容等。


下面以 `Text classification` 中的情感分析任务为例，展示如何使用 Pipeline API。

模型主页：https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english

Hugging Face 在 `pipeline("sentiment-analysis")` 里使用的默认模型，通常是  
**[distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)**。

---

## 1. 这个默认模型是什么？

1. **DistilBERT** 版本：  
   - DistilBERT 是对原始 BERT 做了**模型蒸馏 (distillation)** 后得到的一个轻量化模型，比 BERT 体积更小、推理速度更快，但尽量保留了 BERT 的语言理解能力。

2. **fine-tuned on SST-2**：  
   - 它在 Stanford Sentiment Treebank (SST-2) 数据集上进行了**微调 (fine-tuning)**，用来做**英文情感分类**（比如区分 “正面 / 负面”）。

3. **无大小写 (uncased)**：  
   - 它使用的是“uncased”词表，意味着它不区分大小写（“Apple” 和 “apple” 最终在分词时被当作同一个词处理）。

---

## 2. 它是语言模型吗？

- **BERT/DistilBERT** 本质是 **“预训练语言模型”** 的一种，但它使用 **掩码语言建模（MLM）** 技术进行预训练，并不直接像 GPT 那样生成文本。  
- 不过，这里我们得到的 **“finetuned-sst-2”** 版本已经是针对**情感分类**任务做过专门训练，因此更准确地说，它是**一个针对情感分析下游任务的分类模型**。

换句话说，它的核心还是基于 **BERT 家族** 的预训练语言模型，但是它现在的使用场景是**句子分类**而非通用的文本生成或语言建模。

---

## 3. 使用特点

- **适合英文文本** 的**正/负向情感分析**。对于诸如“good/bad”、“love/hate”之类的表达能判断得比较准确。  
- 如果输入里面包含**中文或其它语言**，这个模型往往无法正确分词、效果也会很差，建议换一个专门针对中文或多语言情感分析的模型。

---

## 4. 总结

1. **默认模型**：`distilbert-base-uncased-finetuned-sst-2-english`  
2. **本质**：基于 BERT (DistilBERT) 的**预训练语言模型**，再进行**情感分类微调**  
3. **使用场景**：英文句子的正面/负面情感分析  
4. **若需中文**：请选择专门的中文情感分析模型（如 `uer/roberta-base-finetuned-jd-binary-chinese` 等）  

因此，如果你在 Python 中直接调用 `pipeline("sentiment-analysis")`，没指定模型，默认就会下载并使用这个 DistilBERT 英文情感分析模型来给你做推理。

**情感分类问题**的期望结果分值应该在 **0.97 以上**，其实这是一个比较高的标准，是否合理需要根据具体的情况来判断。通常来说，分类模型的评估指标是 **准确率 (Accuracy)**、**精确度 (Precision)**、**召回率 (Recall)**、**F1 分数 (F1 Score)** 等。

### 1. **0.97 的准确率标准**：  
对于情感分析任务，0.97 的准确率确实是非常高的。这里有几点要考虑：

- **数据集的难度**：如果是处理相对简单的、清晰标注的情感数据集（例如：SST-2），这种准确率可能更容易达到。但如果是复杂的情感分析任务（例如：讽刺、双关语、隐含情感的理解），0.97 的准确率就相对较难达到。
- **标注的质量和任务的范围**：如果情感分析的任务本身非常精细化（比如分类成多个情感类别：愤怒、喜悦、悲伤等），要达到 0.97 的准确率可能比较困难。相反，如果只是正面和负面（比如二分类任务），那么高准确率是更为容易实现的。
- **数据量**：训练模型所用的数据量越大，通常模型的表现会越好，因此如果模型的训练数据很少，达到 0.97 的准确率可能非常难。

### 2. **情感分析模型的普遍表现**

以 **SST-2** 数据集为例，DistilBERT（你提到的默认模型）在经过微调后的准确率大约能达到 **90% 到 95%** 左右，这已经是非常好的结果了。对于 **BERT** 或 **RoBERTa** 等大模型，可能会稍微提高一些准确率，但一般不会超过 98%。

在实际使用中，准确率并非唯一的衡量标准。**F1 分数**（精确度与召回率的调和平均值）往往更重要，尤其是在样本不均衡时，准确率可能不能全面反映模型的性能。

### 3. **不同任务的目标**

对于情感分析任务，尤其是在一些 **二分类任务**（例如判断一句话是正面还是负面情感）中，目标的准确率有时候并不需要达到 0.97 以上，尤其是在 **生产环境中**，很多时候 90% 左右已经能提供较好的效果，用户可以接受一些误判。相比之下，达到 0.97 的准确率则可能意味着你需要：
- 处理非常简单的文本数据，或者
- 模型进行了非常精细的调优。

### 4. **评估指标的选择**

除了准确率，还有其他重要的指标可以帮助你更全面地了解模型表现：

- **精确度 (Precision)**：模型预测为正类的样本中，真正是正类的比例。如果你对减少假阳性（错误地预测为正类）更为关心，精确度会更重要。
- **召回率 (Recall)**：模型能够识别出所有正类样本的比例。如果你更关心漏掉负面的情感，召回率就是一个需要重点关注的指标。
- **F1 分数**：精确度和召回率的调和平均值，它平衡了精确度和召回率。通常情况下，F1 分数比准确率更能反映模型的真实表现，尤其是在类别不平衡时。

### 5. **结论**

- **0.97 以上的准确率**：对于大部分情感分析任务来说，0.97 的准确率是比较高的标准，可能比较难实现，尤其是任务比较复杂或数据较少的情况下。如果你的模型准确率能达到 0.90-0.95 之间，就已经是非常不错的表现。
- **高标准有道理**：如果你的数据集较简单，且标注质量很高，目标准确率是 0.97 或以上是合理的。但对于更复杂的场景，95% 以上可能已经非常优秀。
- **评估指标综合考虑**：除了准确率，**F1 分数**等其他评估指标也应综合考虑。准确率较高但 F1 分数较低，可能意味着模型存在较多的误分类问题。

综上，**0.97 以上**作为目标并不是不合理，但在许多情感分析任务中，能达到 0.90 到 0.95 之间通常已经是非常不错的表现。

**`distilbert-base-uncased-finetuned-sst-2-english`** 这个模型针对 **英文** 情感分析训练，主要词表（vocabulary）针对的是英文单词，对于中文字符基本没有有效的表示方式。你把中文文本喂给它时，它还是会输出一个结果，但这个结果大概率并没有实际意义，也可以认为“它不认得中文”。

---

## 1. 为什么英文模型不适用于中文
1. **预训练词表差异**：此类英文模型对中文字符（Unicode）没有训练过；当它看到中文时，一般会被强行分词为一堆“[UNK]”（未知标记）或者奇怪的 token，得到的表征几乎没有可用信息。
2. **训练语料差异**：它是在英文句子上做的预训练和微调，对中文的语法、表达并不了解。
3. **情感标注差异**：它只见过英文中正面/负面表述的模式，中文中的“喜怒哀乐”各种表述形式并未学习到。

因此，用它直接做中文文本的情感分析，模型输出结果并不可信。

---

## 2. 想分析中文情感，应该选什么模型？

如果你的目标是做**中文情感分析**，可以考虑**在 Hugging Face** 上找一些已经针对中文进行预训练并且微调过的模型，比如：

- [hfl/chinese-roberta-wwm-ext](https://huggingface.co/hfl/chinese-roberta-wwm-ext) 或 [hfl/chinese-bert-wwm-ext](https://huggingface.co/hfl/chinese-bert-wwm-ext)  
  - 这些是中文语言模型，可以拿来继续做情感分类微调。  
- 一些开箱即用的**中文情感分析**模型，比如 [uer/roberta-base-finetuned-jd-binary-chinese](https://huggingface.co/uer/roberta-base-finetuned-jd-binary-chinese)  
  - 在京东商品评论或其它中文情感语料上微调过，直接可拿来做中文情感分类。

直接换成中文模型后，再输入中文，就能得到**更合理**的结果。

## transformers 自定义模型下载的路径

在transformers自定义模型下载的路径方法

```python
import os

os.environ['HF_HOME'] = '/mnt/new_volume/hf'
os.environ['HF_HUB_CACHE'] = '/mnt/new_volume/hf/hub'
print(os.environ.get("HF_HUB_URL"))
```

In [20]:
import os

os.environ['HF_HOME'] = '/mnt/new_volume/hf'
os.environ['HF_HUB_CACHE'] = '/mnt/new_volume/hf/hub'
print(os.environ.get("HF_HOME"))
print(os.environ.get("HF_HUB_CACHE"))

/mnt/new_volume/hf
/mnt/new_volume/hf/hub


In [21]:
from transformers import pipeline

# 仅指定任务时，使用默认模型（不推荐）
# 默认模型是英文识别，非英文文本都会被判断为NAGATIVE
pipe = pipeline("sentiment-analysis", model="/mnt/new_volume/hf/hub/distilbert-base-uncased-finetuned-sst-2-english")
pipe("今儿上海可真冷啊")

[{'label': 'NEGATIVE', 'score': 0.8957212567329407}]

### 测试更多示例

In [22]:
pipe("我觉得这家店糖醋肉的味道一般")

[{'label': 'NEGATIVE', 'score': 0.8923922181129456}]

In [26]:
# 默认使用的模型 distilbert-base-uncased-finetuned-sst-2-english 
# 并未针对中文做太多训练，中文的文本分类任务表现未必满意
pipe("你学东西真的好快，理论课一讲就明白！")

[{'label': 'NEGATIVE', 'score': 0.7035934925079346}]

In [27]:
# 替换为英文后，文本分类任务的表现立刻改善
pipe("You learn things really quickly. You understand the English class as soon as it is taught.")

[{'label': 'POSITIVE', 'score': 0.9974971413612366}]

In [30]:
pipe("Today Shanghai is very hot.")

[{'label': 'POSITIVE', 'score': 0.9989988207817078}]

### 批处理调用模型推理

In [31]:
text_list = [
    "Today Shanghai is vary cold.",
    "I think the taste of the meat in this store is average.",
    "You learn things really quickly. You understand the japanese class as soon as it is taught."
]

pipe(text_list)

[{'label': 'NEGATIVE', 'score': 0.9976884126663208},
 {'label': 'NEGATIVE', 'score': 0.99625563621521},
 {'label': 'POSITIVE', 'score': 0.998001754283905}]

## 使用 Pipeline API 调用更多预定义任务

## Natural Language Processing(NLP)

**NLP**(自然语言处理)任务是最常见的任务类型之一，因为文本是我们进行交流的一种自然方式。要将文本转换为模型可识别的格式，需要对其进行分词。这意味着将一系列文本划分为单独的单词或子词（标记），然后将这些标记转换为数字。结果就是，您可以将一系列文本表示为一系列数字，并且一旦您拥有了一系列数字，它就可以输入到模型中来解决各种NLP任务！

上面演示的 文本分类任务，以及接下来的标记、问答等任务都属于 NLP 范畴。

### Token Classification

在任何NLP任务中，文本都经过预处理，将文本序列分成单个单词或子词。这些被称为tokens。

**Token Classification**（Token分类）将每个token分配一个来自预定义类别集的标签。

两种常见的 Token 分类是：

- 命名实体识别（NER）：根据实体类别（如组织、人员、位置或日期）对token进行标记。NER在生物医学设置中特别受欢迎，可以标记基因、蛋白质和药物名称。
- 词性标注（POS）：根据其词性（如名词、动词或形容词）对标记进行标记。POS对于帮助翻译系统了解两个相同的单词如何在语法上不同很有用（作为名词的银行与作为动词的银行）。

模型主页：https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english

Hugging Face 的 `pipeline` API 支持多种预定义 NLP 任务，以下是对 Token Classification 及其他常见任务的扩展说明和优化建议：

---

### 一、Token Classification 任务优化
#### 1. **命名实体识别（NER）**
示例中使用的默认模型 `dbmdz/bert-large-cased-finetuned-conll03-english` 是一个在 CoNLL-2003 数据集上微调的模型，适用于英文 NER。建议明确指定模型名称和版本以提升稳定性：
```python
from transformers import pipeline

# 显式指定模型和任务类型
ner_pipeline = pipeline(
    task="ner",
    model="dbmdz/bert-large-cased-finetuned-conll03-english",
    revision="f2482bf"  # 可选，指定模型版本
)
result = ner_pipeline("Apple was founded by Steve Jobs in Cupertino.")
print(result)
```
输出示例：
```text
[{'entity': 'B-ORG', 'score': 0.98, 'word': 'Apple'}, 
 {'entity': 'B-PER', 'score': 0.99, 'word': 'Steve Jobs'}, 
 {'entity': 'B-LOC', 'score': 0.97, 'word': 'Cupertino'}]
```

#### 2. **词性标注（POS）**
若需词性标注，可使用支持多任务的模型（如 `vblagoje/bert-english-uncased-finetuned-pos`）：
```python
pos_pipeline = pipeline(
    task="token-classification",
    model="vblagoje/bert-english-uncased-finetuned-pos",
    aggregation_strategy="simple"  # 合并连续同类标记
)
result = pos_pipeline("The quick brown fox jumps over the lazy dog.")
print(result)
```
输出示例：
```text
[{'entity_group': 'DET', 'word': 'The', 'score': 0.99},
 {'entity_group': 'ADJ', 'word': 'quick', 'score': 0.98},
 ...]
```

---

### 二、其他预定义 NLP 任务示例
#### 1. **文本分类（Text Classification）**
```python
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
result = classifier("Transformers library simplifies NLP tasks!")
# 输出：[{'label': 'POSITIVE', 'score': 0.999}]
```

#### 2. **问答系统（Question Answering）**
```python
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
context = "Hugging Face is a company based in New York."
answer = qa_pipeline(question="Where is Hugging Face located?", context=context)
# 输出：{'answer': 'New York', 'score': 0.92}
```

#### 3. **文本生成（Text Generation）**
```python
generator = pipeline("text-generation", model="gpt2")
text = generator("Artificial intelligence will", max_length=50, num_return_sequences=2)
# 输出两段不同生成的文本
```

#### 4. **文本摘要（Summarization）**
```python
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary = summarizer("Long input text about climate change...", max_length=100)
# 输出摘要后的文本
```

---

### 三、最佳实践建议
1. **显式指定模型**  
   生产环境中应避免依赖默认模型，明确指定模型名称和版本（如 `model="bert-base-uncased"`）。

2. **参数优化**  
   - `aggregation_strategy`：在 NER 中合并连续实体（如 "New York" 合并为单个实体）。
   - `max_length` 和 `truncation`：处理长文本时避免截断错误。

3. **多语言支持**  
   针对非英文任务，选择多语言模型（如 `bert-base-multilingual-cased`）。

4. **性能优化**  
   - 使用 `device` 参数启用 GPU 加速（如 `device=0` 表示使用第一块 GPU）。
   - 批处理输入提升效率：`results = pipeline(texts, batch_size=8)`。

---

### 四、任务扩展场景
| 任务类型               | 典型应用场景                     | 推荐模型示例                          |
|------------------------|----------------------------------|---------------------------------------|
| **命名实体识别（NER）** | 医疗病历解析、法律文件分析       | `dslim/bert-base-NER` (生物医学领域) |
| **情感分析**           | 电商评论分析、舆情监控           | `nlptown/bert-base-multilingual-uncased-sentiment` |
| **文本翻译**           | 多语言内容生成                   | `Helsinki-NLP/opus-mt-zh-en` (中译英) |

---

### 五、引用说明
- 关于 Token Classification 的模型配置和 NER/POS 实现细节可参考[网页2]。
- Pipeline API 的模型加载和参数设置方法详见[网页6]。
- 生产环境中的最佳实践建议来自[网页9]的问答系统配置案例。

通过明确指定模型、调整参数及选择合适任务类型，可充分发挥 Transformers 库的多任务处理能力。

当你在 Hugging Face 的 `pipeline` API 中未显式指定模型时，系统会自动选择一个默认模型。以下是关于此警告的详细解读和优化建议：

---

### 一、警告原因分析
1. **默认模型的局限性**  
   - 当调用 `pipeline(task="ner")` 时未指定模型，Transformers 库会自动选择任务相关的默认模型（如 `dbmdz/bert-large-cased-finetuned-conll03-english`）。
   - **潜在风险**：默认模型可能并非最适合你的任务场景，且不同版本的库可能会调整默认模型，导致结果不一致。

2. **生产环境不推荐**  
   - 警告明确提示 `Using a pipeline without specifying a model name and revision in production is not recommended`，原因包括：
     - **版本控制缺失**：未指定模型版本（`revision`）可能导致未来更新时模型行为变化。
     - **性能不可控**：默认模型的推理速度、内存占用可能不满足生产需求。

---

### 二、解决方案与最佳实践
#### 1. **显式指定模型**
```python
from transformers import pipeline

# 指定模型名称和版本（推荐）
ner_pipeline = pipeline(
    task="ner",
    model="dbmdz/bert-large-cased-finetuned-conll03-english",
    revision="f2482bf"  # 可选，锁定版本
)
```

#### 2. **模型选择策略**
- **任务适配性**：根据任务类型选择专用模型（如医疗 NER 用 `dslim/bert-base-NER`）。
- **性能评估**：优先选择下载量高、社区评价好的模型（Hugging Face Hub 的筛选功能可辅助决策）。
- **多语言支持**：非英文任务需选择多语言模型（如 `bert-base-multilingual-cased`）。

#### 3. **版本控制**
- 通过 `revision` 参数指定模型版本哈希值或分支名（如 `main`），确保代码长期稳定。
- 示例：
  ```python
  pipeline("text-generation", model="gpt2", revision="6c0e608")  # 指定版本哈希
  ```

#### 4. **环境配置优化**
- **缓存管理**：通过 `MODELSCOPE_CACHE` 环境变量自定义模型存储路径（如 `/mnt/new_volume/hf/hub`），避免默认路径空间不足。
- **GPU 加速**：添加 `device=0` 参数启用 GPU 推理（需安装 PyTorch/TensorFlow GPU 版本）。

---

### 三、扩展应用场景
| 任务类型       | 推荐模型                          | 适用场景                     | 关键参数示例                     |
|----------------|-----------------------------------|-----------------------------|----------------------------------|
| **文本分类**   | `distilbert-base-uncased-finetuned-sst-2-english` | 情感分析、垃圾邮件检测       | `return_all_scores=True`         |
| **问答系统**   | `bert-large-uncased-whole-word-masking-finetuned-squad` | 文档检索、客服机器人         | `top_k=3`（返回多个答案）        |
| **文本生成**   | `gpt2` 或 `facebook/bart-large-cnn` | 内容创作、摘要生成           | `max_length=100, temperature=0.7` |

---

### 四、引用说明
- 关于默认模型选择和版本控制问题，详见 Transformers 库的模型加载逻辑。
- 生产环境最佳实践参考了 Hugging Face 官方文档的部署建议。
- 模型选择策略结合了社区经验与 Hub 筛选机制。

通过显式指定模型和版本，可避免默认行为的不确定性，同时提升代码可维护性和推理性能。

In [3]:
from transformers import pipeline

classifier = pipeline(task="ner", model="/mnt/new_volume/hf/hub/models/hengzq/bert-large-cased-finetuned-conll03-english")

Some weights of the model checkpoint at /mnt/new_volume/hf/hub/models/hengzq/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
preds = classifier("Hugging Face is a French company based in New York City.")
preds = [
    {
        "entity": pred["entity"],
        "score": round(pred["score"], 4),
        "index": pred["index"],
        "word": pred["word"],
        "start": pred["start"],
        "end": pred["end"],
    }
    for pred in preds
]
print(*preds, sep="\n")

{'entity': 'I-ORG', 'score': 0.9968, 'index': 1, 'word': 'Hu', 'start': 0, 'end': 2}
{'entity': 'I-ORG', 'score': 0.9293, 'index': 2, 'word': '##gging', 'start': 2, 'end': 7}
{'entity': 'I-ORG', 'score': 0.9763, 'index': 3, 'word': 'Face', 'start': 8, 'end': 12}
{'entity': 'I-MISC', 'score': 0.9983, 'index': 6, 'word': 'French', 'start': 18, 'end': 24}
{'entity': 'I-LOC', 'score': 0.999, 'index': 10, 'word': 'New', 'start': 42, 'end': 45}
{'entity': 'I-LOC', 'score': 0.9987, 'index': 11, 'word': 'York', 'start': 46, 'end': 50}
{'entity': 'I-LOC', 'score': 0.9992, 'index': 12, 'word': 'City', 'start': 51, 'end': 55}


用户提供的代码是在执行 **命名实体识别（NER）任务**，具体功能和警告解析如下：

---

### 一、代码功能解析
#### 1. **模型加载与任务初始化**
```python
classifier = pipeline(
    task="ner",
    model="/mnt/new_volume/hf/hub/models/hengzq/bert-large-cased-finetuned-conll03-english"
)
```
- **任务类型**: 使用 `task="ner"` 初始化一个命名实体识别（NER）的 Pipeline。
- **模型路径**: 加载本地存储的预训练模型 `bert-large-cased-finetuned-conll03-english`（该模型基于 CoNLL-2003 数据集微调，支持识别组织、人名、地点等实体）。

#### 2. **推理与结果处理**
```python
text = "Hugging Face is a French company based in New York City."
preds = classifier(text)
```
- **输入文本**: 模型对句子进行分词和推理，识别文本中的实体。
- **输出示例**:
  ```python
  {'entity': 'I-ORG', 'word': 'Hu', ...}  # "Hugging Face" 被识别为组织
  {'entity': 'I-MISC', 'word': 'French'}  # "French" 被识别为其他类别
  {'entity': 'I-LOC', 'word': 'New York City'}  # 地点识别
  ```
  - **实体类型**: `I-ORG`（组织）、`I-LOC`（地点）、`I-MISC`（其他类别）。
  - **分词细节**: 由于 BERT 类模型使用子词分词，"Hugging" 被拆分为 `Hu` 和 `##gging`，但 Pipeline 会自动合并连续实体。

#### 3. **结果格式化**
```python
preds = [
    {"entity": pred["entity"], "score": round(pred["score"], 4), ...}
    for pred in preds
]
```
- **目的**: 对原始输出进行格式化，提取关键字段（实体类型、置信度、词语位置等），并将置信度四舍五入到小数点后四位。

---

### 二、警告信息解读
#### 1. **警告内容**
```
Some weights of the model checkpoint were not used: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
```
- **原因**: 加载的模型（`BertForTokenClassification`）未使用原始 BERT 模型的 **池化层（Pooler）** 权重。
- **池化层作用**: 在原始 BERT 中，池化层用于生成句子级别的表示（如文本分类任务），但在 Token 分类任务（如 NER）中不需要该层。

#### 2. **是否正常？**
- **正常情况**: 如果模型是从 **其他任务（如文本分类）** 迁移到 Token 分类任务，未使用池化层是预期的。例如，用户加载的模型 `bert-large-cased-finetuned-conll03-english` 是专门针对 NER 任务微调的，因此丢弃了池化层。
- **异常情况**: 如果期望加载的模型与当前任务结构完全一致（例如从 NER 模型加载到 NER 模型），则可能存在问题（需检查模型文件是否损坏）。

---

### 三、扩展建议
#### 1. **优化模型加载**
若需消除警告，可显式指定 `ignore_mismatched_sizes=True`：
```python
from transformers import AutoModelForTokenClassification, AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained(
    "/mnt/new_volume/hf/hub/models/hengzq/...",
    ignore_mismatched_sizes=True
)
tokenizer = AutoTokenizer.from_pretrained("/mnt/new_volume/hf/hub/models/hengzq/...")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
```

#### 2. **实体合并策略**
使用 `aggregation_strategy` 参数合并子词分片的实体：
```python
classifier = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
```
输出示例：
```python
{'entity_group': 'ORG', 'word': 'Hugging Face', 'start': 0, 'end': 12}
{'entity_group': 'MISC', 'word': 'French', 'start': 18, 'end': 24}
{'entity_group': 'LOC', 'word': 'New York City', 'start': 42, 'end': 55}
```

---

### 引用说明
- 关于 NER 模型结构和池化层的作用参考了 Transformers 库的模型加载逻辑（网页2、网页6）。
- 实体合并策略和 Pipeline 配置方法来自 Transformers 官方文档（网页4、网页10）。

池化层（Pooling Layer）是卷积神经网络（CNN）中的一种关键组件，主要用于对输入数据进行下采样操作，以降低数据维度、提取主要特征并增强模型的鲁棒性。以下是其核心要点：

---

### 一、池化层的定义与作用
1. **基本定义**  
   池化层通过滑动固定大小的窗口（如2x2或3x3）对输入特征图进行降采样，保留关键特征的同时减少数据量。这种操作不改变通道数，但会缩小特征图的空间尺寸（如将4x4的特征图变为2x2）。

2. **核心作用**  
   - **降维与计算优化**：通过减少特征图的尺寸，降低后续层的计算量和内存占用。例如，5x5的输入经过步长为2的池化后变为3x3，计算量减少约75%。
   - **特征提取**：保留显著特征（如最大池化选择窗口内的最大值），过滤噪声和冗余信息。
   - **增强模型鲁棒性**：提供平移、旋转和尺度不变性，使模型对输入的小变化不敏感。

---

### 二、池化层的常见类型
1. **最大池化（Max Pooling）**  
   - **操作**：取窗口内的最大值作为输出。例如，窗口\[1, 3, 5, 6\]的最大值为6。  
   - **优势**：保留最显著特征，适用于边缘检测等任务。

2. **平均池化（Average Pooling）**  
   - **操作**：计算窗口内所有值的平均值。例如，窗口\[1, 3, 5, 6\]的平均值为3.75。  
   - **适用场景**：平滑特征图，减少噪声影响。

3. **全局池化（Global Pooling）**  
   - **操作**：将整个特征图压缩为一个值（如全局最大或平均池化），常用于分类任务的最后一层。  
   - **示例**：将14x14的特征图压缩为1x1，直接输入全连接层。

---

### 三、池化层的数学实现与参数
1. **参数设置**  
   - **窗口大小（kernel_size）**：如2x2或3x3。  
   - **步长（stride）**：决定输出尺寸，默认与窗口大小一致（如2x2窗口步长为2时，4x4输入变为2x2输出）。  
   - **填充（padding）**：通常为0，不扩展输入边缘。

2. **输出尺寸公式**  
   \[
   \text{输出尺寸} = \left\lfloor \frac{\text{输入尺寸} - \text{窗口大小} + 2 \times \text{padding}}{\text{步长}} \right\rfloor + 1
   \]
   例如，5x5输入、3x3窗口、步长2、无填充时，输出为2x2。

---

### 四、池化层的优缺点
1. **优点**  
   - **降低过拟合风险**：减少参数数量，抑制模型复杂度。  
   - **高效计算**：降维后减少后续层的计算量。  
   - **特征不变性**：对输入的小平移、旋转具有鲁棒性。

2. **缺点**  
   - **信息丢失**：池化操作可能丢失细节（如平均池化模糊特征）。  
   - **固定操作**：池化规则不可学习，灵活性较低。

---

### 五、应用场景与改进方法
1. **典型应用**  
   - **图像分类**：如ResNet中通过池化层逐步压缩特征图尺寸。  
   - **目标检测**：保留物体的关键位置信息。  
   - **医疗影像分析**：提取病灶区域的显著特征。

2. **改进方法**  
   - **自适应池化（Adaptive Pooling）**：动态调整窗口大小以适应不同输入。  
   - **金字塔池化（Spatial Pyramid Pooling）**：多尺度池化捕捉不同粒度特征。

---

### 六、与其他组件的关联
- **与卷积层的关系**：池化层通常紧随卷积层，用于压缩特征图并增强特征抽象能力。  
- **与全连接层的衔接**：全局池化可将多维特征图压缩为向量，直接输入分类层。

---

### 总结
池化层通过下采样操作在保留关键特征的同时优化计算效率，是CNN中不可或缺的组件。尽管存在信息丢失等问题，但其在模型鲁棒性和计算性能上的优势使其广泛应用于图像处理、目标检测等领域。

#### 合并实体

In [5]:
classifier = pipeline(task="ner", model="/mnt/new_volume/hf/hub/models/hengzq/bert-large-cased-finetuned-conll03-english", grouped_entities=True)
classifier("Hugging Face is a French company based in New York City.")

Some weights of the model checkpoint at /mnt/new_volume/hf/hub/models/hengzq/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/root/miniconda3/envs/peft/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'ORG',
  'score': 0.9674639,
  'word': 'Hugging Face',
  'start': 0,
  'end': 12},
 {'entity_group': 'MISC',
  'score': 0.99828726,
  'word': 'French',
  'start': 18,
  'end': 24},
 {'entity_group': 'LOC',
  'score': 0.99896103,
  'word': 'New York City',
  'start': 42,
  'end': 55}]

### Question Answering

**Question Answering**(问答)是另一个token-level的任务，返回一个问题的答案，有时带有上下文（开放领域），有时不带上下文（封闭领域）。每当我们向虚拟助手提出问题时，例如询问一家餐厅是否营业，就会发生这种情况。它还可以提供客户或技术支持，并帮助搜索引擎检索您要求的相关信息。

有两种常见的问答类型：

- 提取式：给定一个问题和一些上下文，模型必须从上下文中提取出一段文字作为答案、
  模型给定一个问题和一段上下文，然后直接从这段上下文中选取一部分文字作为答案。比如，你问虚拟助手“这家餐厅现在营业吗？”，如果提供了餐厅相关的描述，模型会在文本中找到一段直接回答该问题的信息。
  
- 生成式：给定一个问题和一些上下文，答案是根据上下文生成的；这种方法由`Text2TextGenerationPipeline`处理，而不是下面展示的`QuestionAnsweringPipeline`
  模型给定问题和上下文后，不是直接从文本中选取答案，而是根据上下文生成一个新的回答。这种方法通常由 Text2TextGenerationPipeline 处理，而不是直接用 QuestionAnsweringPipeline。

模型主页：https://huggingface.co/distilbert-base-cased-distilled-squad
distilbert-base-cased-distilled-squad就是一个专门经过微调、用于提取式问答任务的模型。它在 SQuAD 数据集上训练过，能够比较好地从给定文本中抽取出答案。

In [38]:
from transformers import pipeline

question_answerer = pipeline(task="question-answering", model="/mnt/new_volume/hf/hub/models/damotestx/distilbert-base-cased-distilled-squad")

In [39]:
preds = question_answerer(
    question="What is the name of the repository?",
    context="The name of the repository is huggingface/transformers",
)
print(
    f"score: {round(preds['score'], 4)}, start: {preds['start']}, end: {preds['end']}, answer: {preds['answer']}"
)

score: 0.9327, start: 30, end: 54, answer: huggingface/transformers


In [40]:
preds = question_answerer(
    question="What is the capital of China?",
    context="On 1 October 1949, CCP Chairman Mao Zedong formally proclaimed the People's Republic of China in Tiananmen Square, Beijing.",
)
print(
    f"score: {round(preds['score'], 4)}, start: {preds['start']}, end: {preds['end']}, answer: {preds['answer']}"
)

score: 0.9458, start: 115, end: 122, answer: Beijing


### Summarization

**Summarization**(文本摘要）从较长的文本中创建一个较短的版本，同时尽可能保留原始文档的大部分含义。摘要是一个序列到序列的任务；它输出比输入更短的文本序列。有许多长篇文档可以进行摘要，以帮助读者快速了解主要要点。法案、法律和财务文件、专利和科学论文等文档可以摘要，以节省读者的时间并作为阅读辅助工具。

与问答类似，摘要有两种类型：

- 提取式：从原始文本中识别和提取最重要的句子
  这种方法直接从原文中识别出最重要的句子，然后将它们组合成摘要。它类似于从原始文档中剪辑出关键部分，但不会对句子内容进行修改或重新生成。

- 生成式：从原始文本中生成目标摘要（可能包括输入文件中没有的新单词）；`SummarizationPipeline`使用生成式方法
  这种方法则是通过模型生成一个新的摘要，这个摘要不仅仅是从原文中挑选出来的句子，而是模型在理解原文后生成的新的表述，可能包含原文中没有的词汇。Hugging Face 中的 SummarizationPipeline 使用的就是这种生成式方法。

模型主页：https://huggingface.co/t5-base
t5-base是一个经过预训练的生成模型，可以用于生成式摘要以及其他文本生成任务。通过这种方式，可以对长篇文档（如法律、财务文件、专利、科研论文等）进行自动摘要，帮助读者快速捕捉主要内容。

In [9]:
from transformers import pipeline

import os

print("[当前HF_ENDPOINT值]:", os.environ.get('HF_ENDPOINT', '未设置（默认官方地址）'))

summarizer = pipeline(task="summarization",
                      model="t5-base",
                      min_length=8,
                      max_length=32,
)

[当前HF_ENDPOINT值]: https://hf-mirror.com


In [10]:
summarizer(
    """
    In this work, we presented the Transformer, the first sequence transduction model based entirely on attention, 
    replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention. 
    For translation tasks, the Transformer can be trained significantly faster than architectures based on recurrent or convolutional layers. 
    On both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks, we achieve a new state of the art. 
    In the former task our best model outperforms even all previously reported ensembles.
    """
)


[{'summary_text': 'the Transformer is the first sequence transduction model based entirely on attention . it replaces recurrent layers commonly used in encoder-decode'}]

In [11]:
summarizer(
    '''
    Large language models (LLM) are very large deep learning models that are pre-trained on vast amounts of data. 
    The underlying transformer is a set of neural networks that consist of an encoder and a decoder with self-attention capabilities. 
    The encoder and decoder extract meanings from a sequence of text and understand the relationships between words and phrases in it.
    Transformer LLMs are capable of unsupervised training, although a more precise explanation is that transformers perform self-learning. 
    It is through this process that transformers learn to understand basic grammar, languages, and knowledge.
    Unlike earlier recurrent neural networks (RNN) that sequentially process inputs, transformers process entire sequences in parallel. 
    This allows the data scientists to use GPUs for training transformer-based LLMs, significantly reducing the training time.
    '''
)


[{'summary_text': 'large language models (LLMs) are very large deep learning models pre-trained on vast amounts of data . transformers are capable of un'}]

In [12]:
summarizer(
    """
    The rapid development of artificial intelligence (AI) has raised significant ethical concerns. 
    One major issue is algorithmic bias, where AI systems may perpetuate or amplify existing societal biases 
    due to biased training data. For example, facial recognition technologies have shown higher error rates 
    for certain demographic groups. Another challenge is the lack of transparency in deep learning models, 
    often referred to as the 'black box' problem. Policymakers and researchers are advocating for the 
    implementation of AI ethics guidelines, including fairness audits and explainable AI (XAI) frameworks. 
    Additionally, the potential misuse of AI in autonomous weapons systems has sparked international debates 
    on regulation. Addressing these challenges requires collaboration between technologists, ethicists, and 
    legal experts to ensure AI benefits humanity equitably.
    """
)

[{'summary_text': 'the rapid development of artificial intelligence (AI) has raised significant ethical concerns . one major issue is algorithmic bias, where AI systems may perpetuate or'}]


## Audio 音频处理任务

音频和语音处理任务与其他模态略有不同，主要是因为音频作为输入是一个连续的信号。与文本不同，原始音频波形不能像句子可以被划分为单词那样被整齐地分割成离散的块。为了解决这个问题，通常在固定的时间间隔内对原始音频信号进行采样。如果在每个时间间隔内采样更多样本，采样率就会更高，音频更接近原始音频源。

以前的方法是预处理音频以从中提取有用的特征。现在更常见的做法是直接将原始音频波形输入到特征编码器中，以提取音频表示。这样可以简化预处理步骤，并允许模型学习最重要的特征。

### Audio classification

**Audio classification**(音频分类)是一项将音频数据从预定义的类别集合中进行标记的任务。这是一个广泛的类别，具有许多具体的应用，其中一些包括：

- 声学场景分类：使用场景标签（“办公室”、“海滩”、“体育场”）对音频进行标记。
- 声学事件检测：使用声音事件标签（“汽车喇叭声”、“鲸鱼叫声”、“玻璃破碎声”）对音频进行标记。
- 标记：对包含多种声音的音频进行标记（鸟鸣、会议中的说话人识别）。
- 音乐分类：使用流派标签（“金属”、“嘻哈”、“乡村”）对音乐进行标记。

模型主页：https://huggingface.co/superb/hubert-base-superb-er

数据集主页：https://huggingface.co/datasets/superb#er

```
情感识别（ER）为每个话语预测一个情感类别。我们采用了最广泛使用的ER数据集IEMOCAP，并遵循传统的评估协议：我们删除不平衡的情感类别，只保留最后四个具有相似数量数据点的类别，并在标准分割的五折交叉验证上进行评估。评估指标是准确率（ACC）。
```

### 音频处理任务的特点和一个具体的音频分类任务（包括情感识别）的介绍，主要说明如下：

1. 音频信号的连续性

与文本不同，原始音频是连续的波形数据，不能像文字那样简单地切分成单个单词。为了解决这个问题，通常会在固定时间间隔内对音频进行采样，采样率越高，数字化后的音频越接近原始声音。

2. 特征提取方式的演进

以前常用的做法是先预处理音频，提取出手工设计的特征，然后再输入模型。而现在，主流的方法是直接将原始音频波形输入到一个特征编码器中，由模型自动学习出最有用的特征，这简化了预处理步骤。

3. 音频分类任务

音频分类任务是指将一段音频数据标记为预定义的类别，比如：
声学场景分类：例如判断一段录音是在“办公室”、“海滩”还是“体育场”环境中录制的。
声学事件检测：识别音频中的具体事件，如“汽车喇叭声”、“玻璃破碎声”等。
多标签标记：对一段音频中可能存在的多种声音进行标记，比如同时识别会议中的不同说话人或背景声音。
音乐分类：根据音乐风格（例如“金属”、“嘻哈”、“乡村”）给音乐进行分类。

4. 情感识别（ER）

作为一种音频处理任务，情感识别旨在为每个话语预测一个情感类别。这里采用了常用的IEMOCAP数据集，并按照传统评估方法，只保留数据量比较平衡的四个情感类别，使用标准的五折交叉验证，评估指标为准确率（ACC）。

#### 前置依赖包安装

建议在命令行安装必要的音频数据处理包: ffmpeg

```shell
$apt update & apt upgrade
$apt install -y ffmpeg
$pip install ffmpeg ffmpeg-python
```

In [32]:
from transformers import pipeline

classifier = pipeline(task="audio-classification", model="superb/hubert-base-superb-er")

Some weights of the model checkpoint at superb/hubert-base-superb-er were not used when initializing HubertForSequenceClassification: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at superb/hubert-base-superb-er and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametriza

In [33]:
# 使用 Hugging Face Datasets 上的测试文件
preds = classifier("https://hf-mirror.com/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
preds

[{'score': 0.4532, 'label': 'hap'},
 {'score': 0.3622, 'label': 'sad'},
 {'score': 0.0943, 'label': 'neu'},
 {'score': 0.0903, 'label': 'ang'}]

In [34]:
# 使用 Hugging Face Datasets 上的测试文件
preds = classifier("https://hf-mirror.com/datasets/Narsil/asr_dummy/resolve/main/1.flac")
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
preds

[{'score': 0.3642, 'label': 'neu'},
 {'score': 0.3408, 'label': 'hap'},
 {'score': 0.2599, 'label': 'ang'},
 {'score': 0.0351, 'label': 'sad'}]

In [35]:
# 使用本地的音频文件做测试
preds = classifier("data/audio/mlk.flac")
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
preds

[{'score': 0.4532, 'label': 'hap'},
 {'score': 0.3622, 'label': 'sad'},
 {'score': 0.0943, 'label': 'neu'},
 {'score': 0.0903, 'label': 'ang'}]

### Automatic speech recognition（ASR）

**Automatic speech recognition**（自动语音识别）将语音转录为文本。这是最常见的音频任务之一，部分原因是因为语音是人类交流的自然形式。如今，ASR系统嵌入在智能技术产品中，如扬声器、电话和汽车。我们可以要求虚拟助手播放音乐、设置提醒和告诉我们天气。

但是，Transformer架构帮助解决的一个关键挑战是低资源语言。通过在大量语音数据上进行预训练，仅在一个低资源语言的一小时标记语音数据上进行微调，仍然可以产生与以前在100倍更多标记数据上训练的ASR系统相比高质量的结果。

模型主页：https://huggingface.co/openai/whisper-small

下面展示使用 `OpenAI Whisper Small` 模型实现 ASR 的 Pipeline API 示例：

In [36]:
from transformers import pipeline

# 使用 `model` 参数指定模型
transcriber = pipeline(task="automatic-speech-recognition", model="openai/whisper-small")

/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
text = transcriber("https://hf-mirror.com/datasets/Narsil/asr_dummy/resolve/main/1.flac")
text

{'text': ' He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick, peppered, flour-fattened sauce.'}

In [38]:
text = transcriber("data/audio/mlk.flac")
text

{'text': ' I have a dream that one day this nation will rise up and live out the true meaning of its creed.'}

## Computer Vision 计算机视觉

**Computer Vision**（计算机视觉）任务中最早成功之一是使用卷积神经网络（CNN）识别邮政编码数字图像。图像由像素组成，每个像素都有一个数值。这使得将图像表示为像素值矩阵变得容易。每个像素值组合描述了图像的颜色。

计算机视觉任务可以通过以下两种通用方式解决：

- 使用卷积来学习图像的层次特征，从低级特征到高级抽象特征。
- 将图像分成块，并使用Transformer逐步学习每个图像块如何相互关联以形成图像。与CNN偏好的自底向上方法不同，这种方法有点像从一个模糊的图像开始，然后逐渐将其聚焦清晰。

计算机视觉的任务就是让计算机“看懂”图像。最早的成功案例之一是用卷积神经网络（CNN）来识别邮政编码的数字。图像其实就是由一系列像素组成，每个像素都有一个数值代表其颜色，这样图像就可以表示成一个数字矩阵。

解决计算机视觉问题有两种常见方法：

卷积神经网络（CNN）方法
这种方法利用卷积层逐层提取图像特征，从最简单的边缘、纹理（低级特征），到更复杂的形状和物体（高级特征）。这种自底向上的方法类似于从原始图像中一点点“拼凑”出整体的结构和内容。

基于Transformer的方法
这种方法将图像切分成若干小块（patches），然后使用Transformer来学习这些图像块之间的关系。它有点像先看到一个模糊的全局图像，然后逐步将图像中的细节“聚焦”清晰。这种方法更注重全局信息之间的相互联系，与CNN的局部特征提取形成对比。

总结来说，计算机视觉的两种方法各有侧重：CNN善于从局部提取层次特征，而基于Transformer的方法更像是从全局出发，逐步构建图像细节。这两种方法都在不断推动计算机理解和处理图像的能力。

### 卷积神经网络（CNN）
卷积神经网络（CNN）是一种专门用于处理图像和其他网格状数据的深度学习模型。它主要通过卷积操作来提取图像中的局部特征，例如边缘、纹理和形状，再逐层组合这些特征来识别更复杂的模式。CNN的基本构成包括：

- **卷积层**：使用多个小的卷积核在输入图像上滑动，提取局部特征。
- **激活函数**（如ReLU）：引入非线性，使模型能够学习复杂的模式。
- **池化层**：对特征图进行下采样，减少数据维度和计算量，同时保持重要信息。
- **全连接层**：在网络的后部，将提取到的特征整合起来，进行分类或回归任务。

这种结构使CNN在图像识别、目标检测、语音识别等任务中表现优异。

### Image Classificaiton

**Image Classificaiton**(图像分类)将整个图像从预定义的类别集合中进行标记。像大多数分类任务一样，图像分类有许多实际用例，其中一些包括：

- 医疗保健：标记医学图像以检测疾病或监测患者健康状况
- 环境：标记卫星图像以监测森林砍伐、提供野外管理信息或检测野火
- 农业：标记农作物图像以监测植物健康或用于土地使用监测的卫星图像
- 生态学：标记动物或植物物种的图像以监测野生动物种群或跟踪濒危物种

模型主页：https://huggingface.co/google/vit-base-patch16-224

In [ ]:
### 微调适合自己特殊场景的模型

这个思路的基本流程是：

1. **选择预训练模型**：以现有的模型（例如 vit-base-patch16-224）为基础，该模型已经在大规模数据上预训练，学到了通用的视觉特征。

2. **收集并准备数据**：根据你的特殊场景，收集标注好的数据。数据应该反映你应用场景中的具体类别和特点。

3. **微调（Fine-tuning）**：在预训练模型的基础上，用你的场景数据进行微调。通过调整模型参数，使得模型在你的数据上表现更好，适应特定任务。

4. **评估和优化**：在验证集上测试微调后的模型表现，必要时调整超参数和训练策略。

这种方法利用了预训练模型强大的特征提取能力，同时通过微调让模型更好地适应你的实际应用场景，从而实现较高的准确率和鲁棒性。

In [39]:
from transformers import pipeline  # 从 transformers 库中导入 pipeline 函数

# 创建一个图像分类的 pipeline
# 这里会加载一个默认的预训练模型（例如基于 Vision Transformer 的模型）
# 调用 pipeline(task="image-classification") 后，返回的是一个专门用于图像分类任务的 pipeline 对象（通常是一个 ImageClassificationPipeline 实例）。
# 这个对象内部已经加载了默认的预训练模型、图像预处理器和后处理器，并实现了 __call__ 方法，可以直接接收图像输入并输出分类结果。换句话说，它封装了整个图像分类的流程，使得你只需传入图像，就能得到模型的预测结果。
classifier = pipeline(task="image-classification")

No model was supplied, defaulted to google/vit-base-patch16-224 and revision 5dca96d (https://hf-mirror.com/google/vit-base-patch16-224).
Using a pipeline without specifying a model name and revision in production is not recommended.
/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [45]:
# 使用 classifier 对给定 URL 的图像进行分类预测
"""
当调用 `classifier(...)` 时，pipeline 会自动执行以下步骤：  

1. 下载或加载图片（如果是 URL，会先下载图片）；  
2. 对图片进行必要的预处理；  
3. 使用加载好的预训练模型进行推理；  
4. 返回模型预测的结果。  

所以这一行代码执行后，就会直接开始图像识别任务，并返回相应的预测结果。
"""
preds = classifier(
    "https://hf-mirror.com/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)
# 对每个预测结果，将分数四舍五入到小数点后4位，并保留标签信息
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
# 逐行打印预测结果
print(*preds, sep="\n")

{'score': 0.4335, 'label': 'lynx, catamount'}
{'score': 0.0348, 'label': 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor'}
{'score': 0.0324, 'label': 'snow leopard, ounce, Panthera uncia'}
{'score': 0.0239, 'label': 'Egyptian cat'}
{'score': 0.0229, 'label': 'tiger cat'}


![](data/image/cat-chonk.jpeg)

In [41]:
# 使用本地图片（狼猫）
preds = classifier(
    "data/image/cat-chonk.jpeg"
)
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
print(*preds, sep="\n")

{'score': 0.4335, 'label': 'lynx, catamount'}
{'score': 0.0348, 'label': 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor'}
{'score': 0.0324, 'label': 'snow leopard, ounce, Panthera uncia'}
{'score': 0.0239, 'label': 'Egyptian cat'}
{'score': 0.0229, 'label': 'tiger cat'}


![](data/image/panda.jpg)

In [42]:
# 使用本地图片（熊猫）
preds = classifier(
    "data/image/panda.jpg"
)
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
print(*preds, sep="\n")

{'score': 0.9962, 'label': 'giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca'}
{'score': 0.0018, 'label': 'lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens'}
{'score': 0.0002, 'label': 'ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus'}
{'score': 0.0001, 'label': 'sloth bear, Melursus ursinus, Ursus ursinus'}
{'score': 0.0001, 'label': 'brown bear, bruin, Ursus arctos'}


![](data/image/tiger.jpeg)

In [43]:
# 使用本地图片（老虎）
preds = classifier(
    "data/image/tiger.jpeg"
)
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
print(*preds, sep="\n")

{'score': 0.9351, 'label': 'tiger, Panthera tigris'}
{'score': 0.0591, 'label': 'tiger cat'}
{'score': 0.0017, 'label': 'jaguar, panther, Panthera onca, Felis onca'}
{'score': 0.0005, 'label': 'leopard, Panthera pardus'}
{'score': 0.0005, 'label': 'lion, king of beasts, Panthera leo'}


### Object Detection

与图像分类不同，目标检测在图像中识别多个对象以及这些对象在图像中的位置（由边界框定义）。目标检测的一些示例应用包括：

- 自动驾驶车辆：检测日常交通对象，如其他车辆、行人和红绿灯
- 遥感：灾害监测、城市规划和天气预报
- 缺陷检测：检测建筑物中的裂缝或结构损坏，以及制造业产品缺陷

模型主页：https://huggingface.co/facebook/detr-resnet-50

#### 前置依赖包安装

In [25]:
!pip install timm

  Using cached fsspec-2023.12.2-py3-none-any.whl.metadata (6.8 kB)
Using cached fsspec-2023.12.2-py3-none-any.whl (168 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.16.1 requires fsspec[http]<=2023.10.0,>=2023.1.0, but you have fsspec 2023.12.2 which is incompatible.


In [25]:
from transformers import pipeline

detector = pipeline(task="object-detection")

No model was supplied, defaulted to facebook/detr-resnet-50 and revision 2729413 (https://huggingface.co/facebook/detr-resnet-50).
Using a pipeline without specifying a model name and revision in production is not recommended.
/root/miniconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libpng16.so.16: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.mode

In [26]:
preds = detector(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)
preds = [{"score": round(pred["score"], 4), "label": pred["label"], "box": pred["box"]} for pred in preds]
preds

[{'score': 0.9864,
  'label': 'cat',
  'box': {'xmin': 178, 'ymin': 154, 'xmax': 882, 'ymax': 598}}]

![](data/image/cat_dog.jpg)

In [27]:
preds = detector(
    "data/image/cat_dog.jpg"
)
preds = [{"score": round(pred["score"], 4), "label": pred["label"], "box": pred["box"]} for pred in preds]
preds

[{'score': 0.9985,
  'label': 'cat',
  'box': {'xmin': 78, 'ymin': 57, 'xmax': 309, 'ymax': 371}},
 {'score': 0.989,
  'label': 'dog',
  'box': {'xmin': 279, 'ymin': 20, 'xmax': 482, 'ymax': 416}}]

### Homework：替换以上示例中的模型，对比不同模型在相同任务上的性能表现

在 Hugging Face Models 中找到适合你的模型：https://huggingface.co/models